In [7]:
import os
import sys
os.chdir("/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/bin")
sys.path.append(os.getcwd()) 
import torch

from flower.task import Net, train
import torch
# import dataloader
from torch.utils.data import DataLoader
from lib.dataset_utils import *
%matplotlib inline
%load_ext autoreload
%autoreload 2
from lib.physionet import PhysioNet, variable_time_collate_fn, get_data_min_max
from sklearn import model_selection
from torch.utils.data import DataLoader
from lib import utils
from lib.dataset_utils import store_dataset_physionet

path_prefix = "/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data"
prefix = "physionet"
batch_size = 64
classif = False
noise_weight = 0.1
max_t_extrap = 5
n_total_tp = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
amp_start = 1.0
amp_end = 1.0 
freq_start =1.0
freq_end = freq_start
epochs = 5
batch_size = 3
sample_tp = 0.9
n_samples = 4
max_t_extrap = 50
lr_val = 0.01
cut_tp = None
extrap = None

from types import SimpleNamespace
args = SimpleNamespace()
args.sample_tp = 0
args.cut_tp = None
args.extrap = None


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
#################################################################
# Load data
################################################################
train_data_0 = torch.load(f"{path_prefix}/{prefix}/client_0_train.pt")
train_data_1 = torch.load(f"{path_prefix}/{prefix}/client_1_train.pt")
# now they are the same so i can use one
data_min = torch.load(f"{path_prefix}/{prefix}/client_0_data_min.pt")
data_max = torch.load(f"{path_prefix}/{prefix}/client_0_data_max.pt")


/var/folders/yx/rfhrpk093hz1gfwtn7qxt_180000gq/T/ipykernel_75873/192455421.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data_0 = torch.load(f"{path_prefix}/{pre

In [ ]:



train_dataloader = DataLoader(train_data_0, batch_size= batch_size, shuffle=False, 
    collate_fn= lambda batch: variable_time_collate_fn(batch, args, device, data_type = "train",
        data_min = data_min, data_max = data_max))


/var/folders/yx/rfhrpk093hz1gfwtn7qxt_180000gq/T/ipykernel_55284/2981217546.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_min = torch.load(f"{path_prefix}/{prefix

FileNotFoundError: [Errno 2] No such file or directory: '/Users/luisa/Desktop/nygc/cluster/projects/fl4tsf/data/physionet/data_max.pt'

In [21]:
model = Net()
loss_training = train(model, train_dataloader, train_dataloader, 2, lr = lr_val, device = "cpu", loss_per_epoch=True)

Model configuration: {'dataset_name': 'freq_amp_decay_maxfreq_0.1', 'data_folder': '../data/freq_amp_decay_maxfreq_0.1', 'sample_tp': '0.5', 'lr': '0.01', 'lrdecay': '0.99', 'cut_tp': None, 'batch_size': '50', 'extrap': False, 'obsrv_std': '0.01', 'poisson': True, 'rec_layers': '1', 'gen_layers': '1', 'units': '100', 'gru_units': '100', 'latents': '10', 'rec_dims': '20', 'z0_encoder': 'odernn', 'train_classif_w_reconstr': False, 'classif': False, 'linear_classif': False, 'classif_per_tp': False, 'n_labels': '1', 'input_dim': '41', 'aggregation': 'FedAvg', 'storeweights': False, 'alpha': '0.5', 'localepochs': '3', 'serverrounds': '3', 'fractionfit': '1.0', 'fractionevaluate': '1.0', 'device': 'cpu', 'use_wandb': False, 'gradientclipping': False, 'decay_onset': '100'}
20
82
100
Running reconstruction for Latent ODE model
Running reconstruction for Latent ODE model
Running reconstruction for Latent ODE model
Running reconstruction for Latent ODE model
Running reconstruction for Latent ODE

KeyboardInterrupt: 

In [ ]:




total_dataset = torch.load(os.path.join(path_prefix, prefix, "total_data.pt"))
# Shuffle and split

record_id, tt, vals, mask, labels = train_data[0]


n_samples = len(total_dataset)
data_min, data_max = get_data_min_max(total_dataset)
# laod train and test data
train_dataloader = DataLoader(train_data, batch_size= batch_size, shuffle=False, 
    collate_fn= lambda batch: variable_time_collate_fn(batch, args, device, data_type = "train",
        data_min = data_min, data_max = data_max))
test_dataloader = DataLoader(test_data, batch_size = n_samples, shuffle=False, 
    collate_fn= lambda batch: variable_time_collate_fn(batch, args, device, data_type = "test",
        data_min = data_min, data_max = data_max))
model= Net()
dataset_name = "test"

# if not classif:
#     # Concatenate samples from original Train and Test sets
#     # Only 'training' physionet samples are have labels. Therefore, if we do classifiction task, we don't need physionet 'test' samples.
#     total_dataset = total_dataset + test_dataset_obj[:len(test_dataset_obj)]

# total_data_path = os.path.join(path_prefix, prefix, "total_data.pt")
# subset_total = total_dataset[1:1000]
# torch.save(subset_total, total_data_path)

# load tot

# # # save train dataloader 
# train_data_path = os.path.join(path_prefix, prefix, "train_data.pt")
# subset_train = train_data[1:10]
# torch.save(subset_train, train_data_path)
# # save test dataloader
# test_data_path = os.path.join(path_prefix, prefix, "test_data.pt")
# subset_test = test_data[1:10]
# torch.save(subset_test, test_data_path)

#total_dataset = total_dataset + test_dataset_obj[:len(test_dataset_obj)]

# attr_names = train_dataset_obj.params
# data_objects = {"dataset_obj": train_dataset_obj, 
#             "train_dataloader": utils.inf_generator(train_dataloader), 
#             "test_dataloader": utils.inf_generator(test_dataloader),
#             "input_dim": input_dim,
#             "n_train_batches": len(train_dataloader),
#             "n_test_batches": len(test_dataloader),
#             "attr": attr_names, #optional
#             "classif_per_tp": False, #optional
#             "n_labels": 1} #optional


In [2]:
vals.size()

torch.Size([84, 41])

In [ ]:
loss_training = train(model, train_dataloader, test_dataloader, 2, lr = lr_val, device = "cpu", loss_per_epoch=True)

Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Latent ODE model
Computing losses for VAE baseline
Running reconstruction for Lat

/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


KeyboardInterrupt: 

In [ ]:

loader = utils.inf_generator(train_dataloader)

In [12]:
element = next(loader)

In [23]:
e0 = element['observed_data'][0]
e1 = element['observed_data'][1]

In [24]:
e0

tensor([[1.8889e-01, 1.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 9.0909e-04,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 1.6364e-03,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 2.5455e-03,
         0.0000e+00]])

In [ ]:
# example = train_data[4]

# time_steps = example[1]
# data = example[2]
# mask = example[3]

# features =  list(range(len(attr_names)))

# name_features = [ "HR" ]

# for i in features:
#     feature_name = attr_names[i]
#     plt.figure(figsize=(11, 1))
#     # make a scatter plot
#     plt.scatter(time_steps[mask[:, i] == 1], data[mask[:, i] == 1, i], color='red', s=1, label='Observed')
#     # make a line plot for the whole time series but not plottting the zeros 
#     plot_data = data[:, i][mask[:, i] == 1]
#     plt.plot(time_steps[mask[:, i] == 1], plot_data, color='blue', linewidth=0.5, label='Full time series')
#     plt.xlabel('Time (normalized)')
#     plt.ylabel('Value')
#     plt.title(f'{feature_name}')
#     plt.grid()
#     plt.show()

# def run_centralized(train_loader, test_loader, lr = 0.01, epochs = 2, device = "cpu"):
#     model = Net()
#     dataset_name = "test"
#     # train
#     loss_training = train(model, train_loader, test_loader, epochs, lr, device, loss_per_epoch=True)

#     avg_loss, nodesolves, metric_dict = loss_training
#     train_loss = metric_dict["train_loss"]
#     train_mse = metric_dict["train_mse"]
#     val_loss = metric_dict["val_loss"]
#     val_mse = metric_dict["val_mse"]

#     print(f"Nodesolves: {nodesolves}")

#     df = pd.DataFrame({"train_loss": train_loss, "train_mse": train_mse, "val_loss": val_loss, "val_mse": val_mse, "nodesolves": nodesolves})
#     return df

5